## Name: Simple Multi-hop GCN
### Date: 12/4/2024
### Status: Not sure what we wanted to see. 2-hop seems to have more same-class ratio.
### Idea: 
For the time bein we downloaded the benchmark datasets and calculated the same-class ratio, between hops.

### Results:
For Pubmed and CiteSeer, the 2-hop matrix was more informative.


In [20]:
from dgl.data import ChameleonDataset, FlickrDataset, PubmedGraphDataset, AmazonCoBuyPhotoDataset, CiteseerGraphDataset

dataset = CiteseerGraphDataset()#PubmedGraphDataset()

g = dataset[0]

num_classes = dataset.num_classes

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [21]:
import numpy as np
labels = g.ndata["label"]
rows, cols = g.edges()
train_mask = g.ndata['train_mask']
test_mask = g.ndata['test_mask']

A = np.zeros((len(labels), len(labels)))
A[rows.numpy(), cols.numpy()] = 1



In [22]:
import tqdm
import numpy as np
from collections import defaultdict

def get_total_ratio(rows, cols, labels, wanted_inds):
    counts = defaultdict(list)
    degrees = np.bincount(rows)
    for head, tail, degree in zip(rows,cols, degrees):
        val = 0
        if labels[head] == labels[tail]:
            val = 1
        counts[head] += [val]
    return counts


wanted_inds = np.where(train_mask)[0]
ratios = []
counts = []
for p in range(1,10):
    if p == 1:
        A_power = A
    else:
        A_power = np.linalg.matrix_power(A, p)
    
    rows, cols = np.nonzero(A_power[wanted_inds][wanted_inds])
    count = get_total_ratio(rows, cols, labels, wanted_inds)
    counts.append(count)
    ratio = np.mean([sum(v)/len(v) for v in count.values()])
    print(f"Same-class ratio {p}: {100*ratio:.4f}%\n")
    ratios.append(ratio)
    #break

Same-class ratio 1: 70.7724%

Same-class ratio 2: 76.5289%

Same-class ratio 3: 68.0672%

Same-class ratio 4: 62.6050%

Same-class ratio 5: 60.5042%

Same-class ratio 6: 60.0840%

Same-class ratio 7: 59.6639%

Same-class ratio 8: 59.6639%

Same-class ratio 9: 58.4034%



In [23]:
import tqdm
to_check_edges = [index for index, val in enumerate(rows) if val in wanted_inds]
same = 0
for edge_index in tqdm.tqdm_notebook(to_check_edges):
    head, tail = rows[edge_index], cols[edge_index]
    if labels[head] == labels[tail]:
        same += 1
print(same/len(to_check_edges))

/tmp/ipykernel_1804231/1756913912.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for edge_index in tqdm.tqdm_notebook(to_check_edges):


  0%|          | 0/99040 [00:00<?, ?it/s]

0.22864499192245558
